<br><br><br>
# <center>Functional Programming #11</center>




### <center>2018. 7. 27.</center>
<br><br><br>

In [1]:
sealed trait Color
case object Red extends Color
case object Green extends Color
case object Blue extends Color

val colorEq: (Color, Color) => Boolean = {
    case (Red, Red)     => true
    case (Green, Green) => true
    case (Blue, Blue)   => true
    case _              => false
}

colorEq(Red, Red)
colorEq(Green, Green)
colorEq(Red, Green)

defined trait Color
defined object Red
defined object Green
defined object Blue
colorEq: (Color, Color) => Boolean = $sess.cmd0Wrapper$Helper$$Lambda$2975/215588472@178b13ca
res0_5: Boolean = true
res0_6: Boolean = true
res0_7: Boolean = false

In [2]:
def listEq[A]: (List[A], List[A]) => Boolean = {
    case (Nil, Nil)     => true
    case (x::xs, y::ys) => (x == y) && listEq(xs, ys)
    case _              => false
}

listEq(List(1, 2, 3), List(1, 2, 3))
listEq(List(1, 2, 3), List(1, 2))
listEq(List("asdf", "jkl"), List("asdf", "jkl"))
listEq(List("asdf", "jkl"), Nil)

defined function listEq
res1_1: Boolean = true
res1_2: Boolean = false
res1_3: Boolean = true
res1_4: Boolean = false

## <center>Type class</center>
* In computer science, a type class is a type system construct that <font color="blue">supports ***ad hoc polymorphism***</font>.
* This is achieved by <font color="green">adding ***constraints*** to type variables in <u>parametrically polymorphic types</u></font>. Such a constraint typically involves a type class **T** and a type variable **a**, and means that **a** can only be instantiated to a type whose members support the overloaded operations associated with **T**.
<br>
<div style="text-align: right">from https://en.wikipedia.org/wiki/Type_class</div>

# Type class
---

## <center>Eq - Haskell</center>

<pre>
class BasicEq a where
    isEqual :: a -> a -> Bool
</pre>
<br><br>

# Type class
---

## <center>Eq - Haskell</center>

<pre>
instance BasicEq Bool where
    isEqual True  True  = True
    isEqual False False = True
    isEqual _     _     = False
</pre>
<br><br>

# Type class
---

## <center>Eq - Haskell</center>

<pre>
*Main> :t isEqual 
isEqual :: BasicEq a => a -> a -> Bool
</pre>
<br><br>

In [3]:
val asc = Ordering.fromLessThan[Int](_ < _)
val desc = Ordering.fromLessThan[Int](_ > _)

List(3, 4, 2).sorted(asc)
List(3, 4, 2).sorted(desc)

asc: Ordering[Int] = scala.math.Ordering$$anon$9@5f95ed3
desc: Ordering[Int] = scala.math.Ordering$$anon$9@3c1dcfb2
res2_2: List[Int] = List(2, 3, 4)
res2_3: List[Int] = List(4, 3, 2)

In [4]:
implicit val asc = Ordering.fromLessThan[Int](_ < _)

List(3, 4, 2).sorted

asc: Ordering[Int] = scala.math.Ordering$$anon$9@8397d4a
res3_1: List[Int] = List(2, 3, 4)

In [4]:
implicit val asc = Ordering.fromLessThan[Int](_ < _)
implicit val desc = Ordering.fromLessThan[Int](_ > _)

List(3, 4, 2).sorted

cmd4.sc:4: ambiguous implicit values:
 both value desc in class Helper of type => scala.math.Ordering[Int]
 and value asc in class Helper of type => scala.math.Ordering[Int]
 match expected type scala.math.Ordering[Int]
val res4_2 = List(3, 4, 2).sorted
                           ^

: 

In [5]:
List(3, 4, 2).sorted(asc)
List(3, 4, 2).sorted(desc)

res4_0: List[Int] = List(2, 3, 4)
res4_1: List[Int] = List(4, 3, 2)

In [6]:
trait BasicEq[A] {
    def isEqual: A => A => Boolean
}

defined trait BasicEq

In [7]:
implicit val intEq = new BasicEq[Int] {
    def isEqual = x => y => x == y
}

intEq: AnyRef with BasicEq[Int] = $sess.cmd6Wrapper$Helper$$anon$1@3940787

In [8]:
def isEqual[A](x: A)(y: A)(implicit b: BasicEq[A]) = b.isEqual(x)(y)

defined function isEqual

In [9]:
isEqual(2)(3)
isEqual(3)(3)

res8_0: Boolean = false
res8_1: Boolean = true

In [10]:
implicit val listIntEq = new BasicEq[List[Int]] {
    def isEqual = x => y => (x, y) match {
        case (Nil, Nil)     => true
        case (x::xs, y::ys) => x == y && isEqual(xs)(ys)
        case _              => false
    }
}

isEqual(List(1, 2, 3))(List(1, 2, 3))
isEqual(List(1, 2, 3))(List(1, 2))

listIntEq: AnyRef with BasicEq[List[Int]] = $sess.cmd9Wrapper$Helper$$anon$1@271da072
res9_1: Boolean = true
res9_2: Boolean = false

# Higher-order functions
---

## <center>Processing lists</center>


<center>def map[A, B]: (A => B) => List[A] => List[B]</center>
<br><br><br>

In [12]:
def map[A, B]: (A => B) => List[A] => List[B] = f => {
    case Nil   => Nil
    case x::xs => f(x) :: map(f)(xs)
}

val add1: Int => Int = _ + 1
map(add1)(List(1, 2, 3, 4, 5))

map((x: Int) => x + ">>>")(List(1, 2, 3, 4, 5))

defined function map
add1: Int => Int = $sess.cmd11Wrapper$Helper$$Lambda$3357/1004810357@6f8bb5e
res11_2: List[Int] = List(2, 3, 4, 5, 6)
res11_3: List[String] = List("1>>>", "2>>>", "3>>>", "4>>>", "5>>>")

# Algebraic data type
---

## <center>Recursive types - Haskell</center>

<br>
<center><font size="10">data Tree a = Node (Tree a) a (Tree a) | Empty</font></center>
<br><br>

In [13]:
sealed trait Tree[+A]
case class Node[A](l: Tree[A], a: A, r: Tree[A]) extends Tree[A]
case object Empty extends Tree[Nothing]

object Node { def apply[A](l: Tree[A], a: A, r: Tree[A]): Tree[A] = new Node(l, a, r) }

Node(Node(Empty, "left child", Empty),
     "parent",
     Node(Empty, "right child", Empty))

defined trait Tree
defined class Node
defined object Empty
defined object Node
res12_4: Tree[String] = Node(Node(Empty,left child,Empty),parent,Node(Empty,right child,Empty))

In [13]:
val tree = Node((Node(Empty, 1, Node(Empty, 3, Empty))), 4, Node(Empty, 5, Node(Empty, 7, Empty)))

map(add1)(tree)
// def map[A, B]: (A => B) => List[A] => List[B]

cmd13.sc:3: type mismatch;
 found   : cmd13Wrapper.this.cmd12.wrapper.Tree[Int]
 required: List[Int]
val res13_1 = map(add1)(tree)
                        ^

: 

# Generalization
---

## <center>map → fmap</center>

<center>def map[A, B]: (A => B) => List[A] => List[B]</center><br>
<center>def fmap[A, B]: (A => B) => F[A] => F[B]</center>
<br><br>

In [14]:
trait Functor[F[+_]] {
    def fmap[A, B]: (A => B) => F[A] => F[B]
}

defined trait Functor

In [15]:
implicit val treeFunctor = new Functor[Tree] {
    def fmap[A, B] = f => {
        case Empty         => Empty
        case Node(l, a, r) => Node(fmap(f)(l), f(a), fmap(f)(r))
    }
}

treeFunctor: AnyRef with Functor[Tree] = $sess.cmd14Wrapper$Helper$$anon$1@5425650e

In [16]:
def fmap[A, B, F[+A]](f: A => B)(fa: F[A])(implicit e: Functor[F]) = e.fmap(f)(fa)

defined function fmap

In [17]:
val intTree = Node((Node(Empty, 1, Node(Empty, 3, Empty))), 4, Node(Empty, 5, Node(Empty, 7, Empty)))
fmap(add1)(intTree)
fmap(add1)(Empty)
fmap((x: Int) => x + ">>>")(intTree)

val strTree = Node(Node(Empty, "left child", Empty), "parent", Node(Empty, "right child", Empty))
fmap((x: String) => "[" + x + "]")(strTree)
fmap((x: String) => "[" + x + "]")(Empty)

intTree: Tree[Int] = Node(Node(Empty,1,Node(Empty,3,Empty)),4,Node(Empty,5,Node(Empty,7,Empty)))
res16_1: Tree[Int] = Node(Node(Empty,2,Node(Empty,4,Empty)),5,Node(Empty,6,Node(Empty,8,Empty)))
res16_2: Tree[Int] = Empty
res16_3: Tree[String] = Node(Node(Empty,1>>>,Node(Empty,3>>>,Empty)),4>>>,Node(Empty,5>>>,Node(Empty,7>>>,Empty)))
strTree: Tree[String] = Node(Node(Empty,left child,Empty),parent,Node(Empty,right child,Empty))
res16_5: Tree[String] = Node(Node(Empty,[left child],Empty),[parent],Node(Empty,[right child],Empty))
res16_6: Tree[String] = Empty

# Existential types
---

## <center>[_]</center>

In [18]:
val a = Array(1, 2, 3)

def len(l: Array[Any]) = l.length
// len(a)

a: Array[Int] = Array(1, 2, 3)
defined function len

In [19]:
val a = Array(1, 2, 3)

def len[A](l: Array[A]) = l.length
len(a)

a: Array[Int] = Array(1, 2, 3)
defined function len
res18_2: Int = 3

In [20]:
val a = Array(1, 2, 3)

// def len[A](l: Array[A]) = l.length
def len(l: Array[_]) = l.length
len(a)

a: Array[Int] = Array(1, 2, 3)
defined function len
res19_2: Int = 3

In [21]:
implicit val listFunctor = new Functor[List] {
    def fmap[A, B] = f => {
        case Nil   => Nil
        case x::xs => f(x)::fmap(f)(xs)
    }
}

fmap(add1)(List(1, 2, 3, 4, 5))
fmap((x: Int) => x + ">>>")(List(1, 2, 3, 4, 5))

fmap(add1)(List())
fmap(add1)(Nil)
fmap((x: Int) => x + ">>>")(Nil)

listFunctor: AnyRef with Functor[List] = $sess.cmd20Wrapper$Helper$$anon$1@b6b167e
res20_1: List[Int] = List(2, 3, 4, 5, 6)
res20_2: List[String] = List("1>>>", "2>>>", "3>>>", "4>>>", "5>>>")
res20_3: List[Int] = List()
res20_4: List[Int] = List()
res20_5: List[String] = List()

In [22]:
sealed trait Maybe[+A]
case class Just[A](a: A) extends Maybe[A]
case object Nahthing extends Maybe[Nothing]

object Just { def apply[A](a: A): Maybe[A] = new Just(a) }

defined trait Maybe
defined class Just
defined object Nahthing
defined object Just

In [23]:
implicit val maybeFunctor = new Functor[Maybe] {
    def fmap[A, B] = f => {
        case Nahthing => Nahthing
        case Just(a)  => Just(f(a))
    }
}

fmap(add1)(Just(3))
fmap((x: Int) => x + ">>>")(Just(3))
fmap((x: String) => "[" + x + "]")(Just("test"))

maybeFunctor: AnyRef with Functor[Maybe] = $sess.cmd22Wrapper$Helper$$anon$1@3f78e250
res22_1: Maybe[Int] = Just(4)
res22_2: Maybe[String] = Just(3>>>)
res22_3: Maybe[String] = Just([test])

In [24]:
trait Applicative[F[+_]] extends Functor[F] {
    def pure[A]: A => F[A]
    def app[A, B]: (F[A => B]) => F[A] => F[B]
//     def fmap[A, B]: (A => B) => F[A] => F[B] = f => fa => app(pure(f))(fa)
}

defined trait Applicative

In [25]:
implicit val maybeApplicative = new Applicative[Maybe] {
    def pure[A] = Just[A]
    def app[A, B] = ff => fa => (ff, fa) match {
        case (Nahthing, _)  => Nahthing
        case (Just(f), fa)  => fmap(f)(fa)
    }
    def fmap[A, B] = fa => {
        case Nahthing => Nahthing
        case Just(a)  => Just(fa(a))
    }
}

maybeApplicative: AnyRef with Applicative[Maybe] = $sess.cmd24Wrapper$Helper$$anon$1@328b0a7

In [26]:
def app[A, B, F[+A]](ff: F[A => B])(fa: F[A])(implicit e: Applicative[F]) = e.app(ff)(fa)

defined function app

In [27]:
app(Just(add1))(Just(1))
app(Just((x: Int) => x + ">>>"))(Just(1))
app(Just((x: String) => "[" + x + "]"))(Just("test"))

app(Nahthing)(Just(1))
app(Just(add1))(Nahthing)

res26_0: Maybe[Int] = Just(2)
res26_1: Maybe[String] = Just(1>>>)
res26_2: Maybe[String] = Just([test])
res26_3: Maybe[Nothing] = Nahthing
res26_4: Maybe[Int] = Nahthing

In [28]:
def pure[A, F[+A]](a: A)(implicit e: Applicative[F]) = e.pure(a)

defined function pure

In [29]:
val add: Int => Int => Int = x => y => x + y

app(app(pure(add))(Just(3)))(Just(4))
app(app(pure(add))(Just(3)))(Nahthing)
app(app(pure(add))(Nahthing))(Just(4))

add: Int => Int => Int = $sess.cmd28Wrapper$Helper$$Lambda$3715/1124508918@4c8ece58
res28_1: Maybe[Int] = Just(7)
res28_2: Maybe[Int] = Nahthing
res28_3: Maybe[Int] = Nahthing